In [1]:
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer

/root/miniconda3/envs/pissaquant/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = AutoModelForCausalLM.from_pretrained("/data/mfx/huggingface/meta-llama/Llama-3.2-1B", device_map='cuda')
tokenizer = AutoTokenizer.from_pretrained("/data/mfx/huggingface/meta-llama/Llama-3.2-1B")

In [3]:
inputs = tokenizer("Here I'll write a poem about the sea.", return_tensors="pt").to("cuda")

In [4]:
output = model.generate(**inputs, max_new_tokens=128, do_sample=False)
print(tokenizer.batch_decode(output)[0])

/root/miniconda3/envs/pissaquant/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/miniconda3/envs/pissaquant/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


<|begin_of_text|>Here I'll write a poem about the sea. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of


In [5]:
model.model.layers[0].self_attn.q_proj.weight

Parameter containing:
tensor([[-0.0183,  0.0071,  0.0219,  ..., -0.0070, -0.0089,  0.0149],
        [ 0.0112,  0.0593,  0.0630,  ..., -0.0334, -0.0148,  0.0058],
        [ 0.0182,  0.0141,  0.0361,  ..., -0.0432, -0.0388, -0.0233],
        ...,
        [ 0.0305,  0.0289,  0.0801,  ..., -0.0767, -0.0311, -0.0334],
        [ 0.0242, -0.0325,  0.0369,  ..., -0.0123, -0.0269, -0.0151],
        [-0.0264, -0.0498, -0.0210,  ...,  0.0601,  0.0130, -0.0007]],
       device='cuda:0', requires_grad=True)

In [6]:
lora_config = LoraConfig(r=32, target_modules=['q_proj'], pissaquant_config={'pissaquant_bits': 8, 'apply_quantization': True}, init_lora_weights='PiSSAQuant')

In [7]:
lora_config.pissaquant_config['pissaquant_bits']

8

In [8]:
peft_model = get_peft_model(model, lora_config)

In [9]:
peft_model.base_model.model.model.layers[0].self_attn.q_proj.lora_A['default'].weight

Parameter containing:
tensor([[ 0.1512,  0.1512,  0.1512,  ...,  0.0600,  0.0600,  0.0600],
        [-0.0029, -0.0029, -0.0029,  ...,  0.1133,  0.1133,  0.1133],
        [-0.0241, -0.0241, -0.0241,  ...,  0.0592,  0.0592,  0.0592],
        ...,
        [ 0.1102,  0.1102,  0.1102,  ...,  0.0794,  0.0794,  0.0794],
        [-0.0198, -0.0198, -0.0198,  ..., -0.0679, -0.0679, -0.0679],
        [ 0.0415,  0.0415,  0.0415,  ...,  0.1469,  0.1469,  0.1469]],
       device='cuda:0', requires_grad=True)

In [10]:
output = peft_model.generate(**inputs, max_new_tokens=128, do_sample=False)
print(tokenizer.batch_decode(output)[0])

/root/miniconda3/envs/pissaquant/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/miniconda3/envs/pissaquant/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|>Here I'll write a poem about the sea. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of


In [ ]:
# model = peft_model.merge_and_unload()
# output = model.generate(**inputs, max_new_tokens=128, do_sample=False)
# print(tokenizer.batch_decode(output)[0])

In [ ]:
# model.model.layers[0].self_attn.q_proj.weight

Parameter containing:
tensor([[-0.0184,  0.0071,  0.0218,  ..., -0.0069, -0.0089,  0.0148],
        [ 0.0111,  0.0598,  0.0630,  ..., -0.0332, -0.0146,  0.0057],
        [ 0.0183,  0.0143,  0.0359,  ..., -0.0430, -0.0386, -0.0234],
        ...,
        [ 0.0306,  0.0289,  0.0798,  ..., -0.0765, -0.0307, -0.0332],
        [ 0.0243, -0.0324,  0.0368,  ..., -0.0124, -0.0269, -0.0150],
        [-0.0260, -0.0498, -0.0210,  ...,  0.0595,  0.0132, -0.0009]],
       device='cuda:0')

In [13]:
peft_model.peft_config['default'].init_lora_weights = "PiSSAQuant_load"
peft_model.save_pretrained("PiSSAQuant-Llama-3.2-1B/pissaquant_init")
model = peft_model.unload()
model.save_pretrained("PiSSAQuant-Llama-3.2-1B")
tokenizer.save_pretrained("PiSSAQuant-Llama-3.2-1B")

[2025-03-15 14:07:57,128] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/data/mfx/miniconda3/envs/pissaquant/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/data/mfx/miniconda3/envs/pissaquant/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/pissaquant/lib/libcufile.so: undefined reference to `dlvsym'
/data/mfx/miniconda3/envs/pissaquant/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/pissaquant/lib/libcufile.so: undefined reference to `dlopen'
/data/mfx/miniconda3/envs/pissaquant/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/pissaquant/lib/libcufile.so: undefined reference to `dlclose'
/data/mfx/miniconda3/envs/pissaquant/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/pissaquant/lib/

('PiSSAQuant-Llama-3.2-1B/tokenizer_config.json',
 'PiSSAQuant-Llama-3.2-1B/special_tokens_map.json',
 'PiSSAQuant-Llama-3.2-1B/tokenizer.json')

In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("PiSSAQuant-Llama-3.2-1B", device_map='cuda')
tokenizer = AutoTokenizer.from_pretrained("PiSSAQuant-Llama-3.2-1B")
from peft import PeftModel
peft_model = PeftModel.from_pretrained(model, "PiSSAQuant-Llama-3.2-1B/pissaquant_init")

In [15]:
output = peft_model.generate(**inputs, max_new_tokens=128, do_sample=False)
print(tokenizer.batch_decode(output)[0])

/root/miniconda3/envs/pissaquant/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/root/miniconda3/envs/pissaquant/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<|begin_of_text|>Here I'll write a poem about the sea. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of a haiku. I'll write it in the form of


In [16]:
peft_model.base_model.model.model.layers[0].self_attn.q_proj.lora_A['default'].weight

Parameter containing:
tensor([[ 0.1512,  0.1512,  0.1512,  ...,  0.0600,  0.0600,  0.0600],
        [-0.0029, -0.0029, -0.0029,  ...,  0.1133,  0.1133,  0.1133],
        [-0.0241, -0.0241, -0.0241,  ...,  0.0592,  0.0592,  0.0592],
        ...,
        [ 0.1102,  0.1102,  0.1102,  ...,  0.0794,  0.0794,  0.0794],
        [-0.0198, -0.0198, -0.0198,  ..., -0.0679, -0.0679, -0.0679],
        [ 0.0415,  0.0415,  0.0415,  ...,  0.1469,  0.1469,  0.1469]],
       device='cuda:0')